<a href="https://colab.research.google.com/github/pranabsarkar/Enlish-to-German-Translator-sing-seq2seq-model/blob/master/eng_german.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Colab Notebooks/german/english"

deu.txt			 english-german-test.pkl   model.h5
english-german-both.pkl  english-german-train.pkl
english-german.pkl	 german.ipynb


In [0]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot

Selecting previously unselected package fontconfig.
(Reading database ... 22298 files and directories currently installed.)
Preparing to unpack .../00-fontconfig_2.12.6-0ubuntu2_amd64.deb ...
Unpacking fontconfig (2.12.6-0ubuntu2) ...
Selecting previously unselected package libann0.
Preparing to unpack .../01-libann0_1.1.2+doc-6_amd64.deb ...
Unpacking libann0 (1.1.2+doc-6) ...
Selecting previously unselected package libcdt5.
Preparing to unpack .../02-libcdt5_2.40.1-2_amd64.deb ...
Unpacking libcdt5 (2.40.1-2) ...
Selecting previously unselected package libcgraph6.
Preparing to unpack .../03-libcgraph6_2.40.1-2_amd64.deb ...
Unpacking libcgraph6 (2.40.1-2) ...
Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../04-libjbig0_2.1-3.1build1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1build1) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../05-libtiff5_4.0.9-5_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.9-5) ...
Selecting pr

**Data Cleaning**

In [0]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# normalize unicode characters
			line = normalize('NFD', line).encode('ascii', 'ignore')
			line = line.decode('UTF-8')
			# tokenize on white space
			line = line.split()
			# convert to lowercase
			line = [word.lower() for word in line]
			# remove punctuation from each token
			line = [word.translate(table) for word in line]
			# remove non-printable chars form each token
			line = [re_print.sub('', w) for w in line]
			# remove tokens with numbers in them
			line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
filename = '/content/drive/My Drive/Colab Notebooks/german/english/deu.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, '/content/drive/My Drive/Colab Notebooks/german/english/english-german.pkl')
# spot check
for i in range(100):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: /content/drive/My Drive/Colab Notebooks/german/english/english-german.pkl
[hi] => [hallo]
[hi] => [gru gott]
[run] => [lauf]
[wow] => [potzdonner]
[wow] => [donnerwetter]
[fire] => [feuer]
[help] => [hilfe]
[help] => [zu hulf]
[stop] => [stopp]
[wait] => [warte]
[go on] => [mach weiter]
[hello] => [hallo]
[i ran] => [ich rannte]
[i see] => [ich verstehe]
[i see] => [aha]
[i try] => [ich probiere es]
[i won] => [ich hab gewonnen]
[i won] => [ich habe gewonnen]
[smile] => [lacheln]
[cheers] => [zum wohl]
[freeze] => [keine bewegung]
[freeze] => [stehenbleiben]
[got it] => [kapiert]
[got it] => [verstanden]
[got it] => [einverstanden]
[he ran] => [er rannte]
[he ran] => [er lief]
[hop in] => [mach mit]
[hug me] => [druck mich]
[hug me] => [nimm mich in den arm]
[hug me] => [umarme mich]
[i fell] => [ich fiel]
[i fell] => [ich fiel hin]
[i fell] => [ich sturzte]
[i fell] => [ich bin hingefallen]
[i fell] => [ich bin gesturzt]
[i know] => [ich wei]
[i lied] => [ich habe gelogen]
[i l

**Preparing the data**

In [0]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('/content/drive/My Drive/Colab Notebooks/german/english/english-german.pkl')

# reduce dataset size
n_sentences = 10000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:9000], dataset[9000:]
# save
save_clean_data(dataset, '/content/drive/My Drive/Colab Notebooks/german/english/english-german-both.pkl')
save_clean_data(train, '/content/drive/My Drive/Colab Notebooks/german/english/english-german-train.pkl')
save_clean_data(test, '/content/drive/My Drive/Colab Notebooks/german/english/english-german-test.pkl')

Saved: /content/drive/My Drive/Colab Notebooks/german/english/english-german-both.pkl
Saved: /content/drive/My Drive/Colab Notebooks/german/english/english-german-train.pkl
Saved: /content/drive/My Drive/Colab Notebooks/german/english/english-german-test.pkl


**Defining the model**

In [0]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
import pydot

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

# load datasets
dataset = load_clean_sentences('/content/drive/My Drive/Colab Notebooks/german/english/english-german-both.pkl')
train = load_clean_sentences('/content/drive/My Drive/Colab Notebooks/german/english/english-german-train.pkl')
test = load_clean_sentences('/content/drive/My Drive/Colab Notebooks/german/english/english-german-test.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))

# prepare training data
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_output(trainY, ger_vocab_size)
# prepare validation data
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_output(testY, ger_vocab_size)

# define model
model = define_model(eng_vocab_size, ger_vocab_size, eng_length, ger_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())



English Vocabulary Size: 2309
English Max Length: 5
German Vocabulary Size: 3657
German Max Length: 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 256)            591104    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 10, 256)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 10, 3657)          939849    
Total params: 2,581,577
Trainable params: 2,581,577
Non-trainable params: 0
_________________________________________________________________
None


**Model Training**

In [0]:
# fit model
filename = '/content/drive/My Drive/Colab Notebooks/german/english/model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Train on 9000 samples, validate on 1000 samples
Epoch 1/30
 - 12s - loss: 3.1259 - val_loss: 2.2375

Epoch 00001: val_loss improved from inf to 2.23754, saving model to /content/drive/My Drive/Colab Notebooks/german/english/model.h5
Epoch 2/30
 - 10s - loss: 2.1379 - val_loss: 2.1138

Epoch 00002: val_loss improved from 2.23754 to 2.11375, saving model to /content/drive/My Drive/Colab Notebooks/german/english/model.h5
Epoch 3/30
 - 10s - loss: 2.0368 - val_loss: 2.0684

Epoch 00003: val_loss improved from 2.11375 to 2.06837, saving model to /content/drive/My Drive/Colab Notebooks/german/english/model.h5
Epoch 4/30
 - 10s - loss: 1.9613 - val_loss: 2.0085

Epoch 00004: val_loss improved from 2.06837 to 2.00846, saving model to /content/drive/My Drive/Colab Notebooks/german/english/model.h5
Epoch 5/30
 - 10s - loss: 1.8714 - val_loss: 1.9227

Epoch 00005: val_loss improved from 2.00846 to 1.92267, saving model to /content/drive/My Drive/Colab Notebooks/german/english/model.h5
Epoch 6/30


**Model Evalution**

In [0]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model


# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, ger_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_target, raw_src, translation))
      
      
		

  
 
  

# load datasets
dataset = load_clean_sentences('/content/drive/My Drive/Colab Notebooks/german/english/english-german-both.pkl')
train = load_clean_sentences('/content/drive/My Drive/Colab Notebooks/german/english/english-german-train.pkl')
test = load_clean_sentences('/content/drive/My Drive/Colab Notebooks/german/english/english-german-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
testX = encode_sequences(eng_tokenizer, eng_length, test[:, 0])

# load model
model = load_model('/content/drive/My Drive/Colab Notebooks/german/english/model.h5')
# test on some training sequences
print('Evalution on the train data')
evaluate_model(model, ger_tokenizer, trainX, train)
# test on some test sequences
print('Evalution on the test data')
evaluate_model(model, ger_tokenizer, testX, test)

Evalution on the train data
src=[tom swims], target=[tom schwimmt], predicted=[tom schwimmt]
src=[mary looks hot], target=[maria sieht hei aus], predicted=[maria sieht aus aus]
src=[are we safe], target=[sind wir sicher], predicted=[sind wir uns]
src=[grab my hand], target=[nimm meine hand], predicted=[nimm meine]
src=[i have won], target=[ich gewann], predicted=[ich habe sieger]
src=[lets try it], target=[lasst es uns versuchen], predicted=[versuchen wir es]
src=[good work tom], target=[gute arbeit tom], predicted=[gute braucht tom]
src=[keep me posted], target=[halten sie mich auf dem laufenden], predicted=[halten sie mich zu]
src=[tom is sitting], target=[tom sitzt], predicted=[tom sitzt]
src=[count on it], target=[verlass dich drauf], predicted=[verlass dich drauf]
Evalution on the test data
src=[tom is vain], target=[tom ist eingebildet], predicted=[tom ist eitel]
src=[you look sick], target=[du siehst krank aus], predicted=[du wirkst krank]
src=[youre too big], target=[du bist zu